# (Q)分子構造+(R)理由+(A)物性データセットのLLMによる学習と予測
- Q&A: 融点データセットを使用
- R: 自分自身で考えさせて､正解のデータを学習させる

In [1]:
%reload_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from transformers import AutoTokenizer
import pandas as pd
import random
import copy
import glob
import json
from datetime import datetime
from llmchem.utils import mk_dir,clean_vram

#import clear_output

from IPython.display import clear_output

/home/setup/miniconda3/envs/chemllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#dataset settings
n_test=50 #number of testing data
n_train_check=50 #number of training data for checking (i.e., checking everything takes too long, so we check only a part of training data)
n_GPT_reasoning=10 # number of reasoning data made by GPT
n_generation_iterations=100   # trial numbers to generate new self reasoning data
max_generations=50

#model settings
model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"
target_modules= [
    "lm_head",
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
    "gate",
    "w1",
    "w2",
    "w3"
]

model_name=f"meta-llama/Llama-2-7b-chat-hf"
target_modules= [
    #"embed_tokens",
    "lm_head",
    #"q_proj",
    #"k_proj",
    "v_proj",
    "o_proj",
    "gate_proj",
    "up_proj",
    #"down_proj",
]



#LoRA settings
r=32
lora_alpha=r
bit=16
#bit=8
#bit=4

#train settings
gradient_checkpointing =False
per_device_train_batch_size=1
epochs=3
lr=10**-5

#device settings
device_map="auto"

#dataset path
dataset_path="dataset/231225AutoReasoning/240117best_reason_record_3k.csv"

#project path
project_dir="results/projects/240117llama7b_10"

#reasoning options
error_threshold=30  # if abolute error is smaller than this, add to training data

In [3]:
mk_dir(project_dir)
mk_dir(project_dir+"/eval")
mk_dir(project_dir+"/self_reasoning")
mk_dir(project_dir+"/train")

In [4]:
from llmchem.model import init_model
from llmchem.train import train_model
from llmchem.eval import eval_model
from llmchem.reasoning import self_reasoning
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [5]:
#load base dataset

df=pd.read_csv(dataset_path)
dataset=df.to_dict(orient="records")
random.seed(0)
random.shuffle(dataset)

base_train_dataset=dataset[:-n_test]
example_reasoning_dataset=base_train_dataset[:n_GPT_reasoning]
test_dataset=dataset[-n_test:]

In [6]:
#Loop: training, evaluation, data generation
for generation in range(max_generations):
    clear_output()
    print(f"Generation: {generation}")
    #prepare train dataset

    ## reason data made by GPT4
    train_dataset=copy.deepcopy(example_reasoning_dataset)

    print(f"GPT-generated reasons: {len(train_dataset)}")

    ## reason data made by model itself
    for path in glob.glob(f"{project_dir}/self_reasoning/*.json"):
        with open(path) as f:
            train_dataset.append(json.load(f))

    print(f"All-generated reasons: {len(train_dataset)}")
    random.shuffle(train_dataset)

    #train model
    clean_vram()
    model=init_model(model_name, r, lora_alpha, target_modules, bit=bit,device_map=device_map)
    train_result=train_model(model,tokenizer,train_dataset,
                    project_dir=project_dir,
                    epochs=epochs,
                    lr=lr,
                    per_device_train_batch_size=per_device_train_batch_size,
                    gradient_checkpointing=gradient_checkpointing,
                    )

    #eval

    train_check_dataset=copy.deepcopy(train_dataset[:n_train_check])
    random.shuffle(train_check_dataset)
    train_eval_result=eval_model(model,tokenizer,train_check_dataset,
                                f"{project_dir}/eval",
                                n_prompt_examples=3,
                                prompt_dataset=example_reasoning_dataset,
                                prefix=f"train_{generation}"
                                )

    test_eval_result=eval_model(model,tokenizer,test_dataset,
                                f"{project_dir}/eval",
                                n_prompt_examples=3,
                                prompt_dataset=example_reasoning_dataset,
                                prefix=f"test_{generation}"
                                )

    #generate additional training data by self-reasoning
    self_reasoning(model,tokenizer,base_train_dataset,
                example_reasoning_dataset,project_dir,
                generation=generation,
                n_iterations=n_generation_iterations,
                error_threshold=error_threshold,
                n_max_trials=2)

Generation: 10
GPT-generated reasons: 10
All-generated reasons: 185
Using fp16 mode


Map: 100%|██████████| 185/185 [00:00<00:00, 11239.25 examples/s]


Step,Training Loss
100,1.448000
200,0.964500
300,0.791500
400,0.720500
500,0.666000


  0%|          | 0/50 [00:00<?, ?it/s]

promlem 1 / 50


...

----


##Prediction: 45.0


#Problem


 53%|█████▎    | 53/100 [31:16<29:09, 37.23s/it]

----


The presence of a phenyl group in the base structure of the compound contributes to a higher melting point due to increased molecular weight and potential for pi-pi stacking. The sulfoxide group also contributes to a higher melting point due to its polar nature and capability to form strong dipole-dipole interactions. The presence of two chlorine atoms adds to the molecular weight and polarity, which can increase the melting point. The vinyl group can also contribute to a decrease in melting point due to its rigidity and potential for pi-pi stacking. The presence of a phenyl group in the vinyl group can also increase the melting point due to increased molecular weight and potential for pi-pi stacking. The overall effect of each group must be considered synergistically, rather than additively, due to the complex interplay of implied intermolecular interactions.
##Prediction: 133.0


#Problem
actual:  109.85 predicted:  133.0 err:  23.150000000000006
The presence of a phenyl group

 54%|█████▍    | 54/100 [32:15<33:37, 43.87s/it]

----


The base unit thiophene has a melting point of -2°C. The presence of a carboxylic acid group typically raises the melting point due to increased molecular weight and increased intermolecular interactions, such as hydrogen bonding. The chlorine atom adds a polar and electronegative group, which can increase the melting point due to increased molecular polarity and potential hydrogen bonding. Estimating a contribution of approximately +50°C for the carboxylic acid group and +20°C for the chlorine atom.
##Prediction: 57.0


#Problem
actual:  151.0 predicted:  57.0 err:  94.0
The base unit thiophene has a melting point of -2°C. The presence of a carboxylic acid group typically raises the melting point due to increased molecular weight and increased intermolecular interactions, such as hydrogen bonding. The chlorine atom adds a polar and electronegative group, which can increase the melting point due to increased molecular polarity and potential hydrogen bonding. Estimating a contrib